In [ ]:
import numpy as np
import pandas as pd

df1=pd.read_csv('MTA_Subway_HourlyBeginning_February_2022.csv')
df2=pd.read_csv('subway2022jan.csv')
df1.info()

In [ ]:
df2.info()
df=pd.concat([df1,df2],ignore_index=True)

In [ ]:
df.info()

In [ ]:
#df['borough'].value_counts()
df=df.drop(columns=['transit_mode', 'fare_class_category'])
df.info()

In [ ]:
org_df=df.copy()
df['borough'].value_counts()

In [ ]:
mapping={'BK':'Brooklyn','M':'Manhattan','BX':'Bronx','Q':'Queens'}
df['borough']=df['borough'].replace(mapping)

In [ ]:
df['borough'].unique()

In [ ]:
df['borough'].value_counts()
station_df=df[['station_complex','borough']]
station_df.groupby('borough')['station_complex'].nunique()

In [ ]:
df['transit_timestamp']=df['transit_timestamp'].str.split(' ')

In [ ]:
df['transit_timestamp'][100]

In [ ]:
df['date']=[x[0] for x in df['transit_timestamp']]
df['time']=[x[1]+" "+x[2] for x in df['transit_timestamp']]

In [ ]:
df.drop(axis='columns',labels='transit_timestamp',inplace=True)

In [ ]:
df.info()

In [ ]:
df['date']=pd.to_datetime(df['date'],format="%m/%d/%Y")
df['time']=pd.to_datetime(df['time'],format='%I:%M:%S %p').dt.time

In [ ]:
df['full_datetime']=pd.to_datetime(df['date'].astype(str)+' '+df['time'].astype(str),errors='coerce')

In [ ]:
df2022=df[(df['full_datetime']>=pd.to_datetime('2022-01-01'))&(df['full_datetime']<=pd.to_datetime('2023-01-01'))]

In [ ]:
df2022['full_datetime'].min()
df2022['full_datetime'].max()
df2022.duplicated().sum()

In [ ]:
#df2022.drop(axis='columns',labels=['time'],inplace=True)
df2022.duplicated(subset=['station_complex_id']).sum()
df2022[df2022.duplicated(subset=['station_complex_id', 'full_datetime'])].head(5)

In [ ]:
df2022['check_key']=df2022['station_complex_id'].astype(str)+'_'+df2022['full_datetime'].astype(str)+df2022['time'].astype(str)

In [ ]:
#df2022['month']=df2022['full_datetime'].dt.month
df2022[df2022['check_key']=='R639_2022-06-05 04:00:0004:00:00']

In [ ]:
df2022.loc[df2022['check_key'] =='R639_2022-06-05 04:00:0004:00:00']
duplicate_indexes=df2022.index[df2022['check_key']=='R639_2022-06-05 04:00:0004:00:00'].tolist()
print(duplicate_indexes)

In [ ]:
#import calendar
#monthnames=list(calendar.month_name)
#monthnames
#df2022['month']=df2022['month'].map(lambda x: monthnames[x])
#df2022['month'].value_counts()

In [ ]:
#df2022['full_datetime'].min()

In [ ]:
#df2022['year']=df2022['full_datetime'].dt.year

In [ ]:
#df2022.head(5)

In [ ]:
df2022['check_key'].value_counts().reset_index(name='count').rename(columns={'index': 'check_key'})

In [ ]:
#no duplicate- ridership numbers splitted by also payment_method
df2022[df2022['check_key']=='611_2022-01-24 14:00:0014:00:00']

In [ ]:
grouped_df=df2022.groupby(['station_complex_id','station_complex','borough','date','time','full_datetime']).agg({
    'ridership': 'sum',
    'latitude': 'first',
    'longitude': 'first',
    'Georeference': 'first'}).reset_index()

In [ ]:
grouped_df.info()

In [ ]:
grouped_df.sort_values(by='ridership',ascending=False).head(5)

In [ ]:
grouped_df['date'].max()
copyofgrouped=grouped_df.copy()

In [ ]:
#grouped_df.to_csv('groupedcleanedsubway22.csv')

In [ ]:
#df.groupby('station_complex').count()
grouped_df

In [ ]:
import calendar
monthnames=list(calendar.month_name)
monthnames
grouped_df['month']=grouped_df['date'].dt.month_name()
grouped_df['month']=grouped_df['month'].map(lambda x: monthnames[x])
grouped_df['month'].value_counts()

In [ ]:
grouped_df['month'].unique()

In [ ]:
grouped_df.info()

In [ ]:
dfyear=grouped_df.groupby(['station_complex_id','station_complex','borough','latitude','longitude'],as_index=False)['ridership'].sum()


In [ ]:
dfyear.info()
dfyear['ridership'].sum()

In [ ]:
import plotly.express as px

fig = px.density_mapbox(
    dfyear.sort_values(by='ridership', ascending=False).head(500),
    lat='latitude',
    lon='longitude',
    z='ridership',
    hover_name='station_complex',
    radius=20,
    zoom=10,
    mapbox_style='open-street-map',
    height=700,
    title='NYC Subway Stations by Annual Ridership (2022) — Density Heatmap'
)
fig.show()

In [ ]:
dfyear.sort_values(by='ridership',ascending=False).head(10)

In [ ]:
manhattantotalnum=dfyear[dfyear['borough']=='Manhattan']['ridership']

In [ ]:
manhattantotalnum.sum()

In [ ]:
borough_df = borough_total_df[:4].reset_index()
borough_total_df=dfyear.groupby('borough')['ridership'].sum()

In [ ]:
borough_total_df[:4]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

top10stationsorted=dfyear.sort_values(by='ridership',ascending=False).head(10)
plt.figure(figsize=(10,6))
sns.barplot(data=top10stationsorted,x='ridership',y='station_complex',palette='viridis',orient='h')


plt.ylabel('Station Names',fontsize=14)
plt.title('Top 10 Subway Stations by Annual 2022 Ridership Figures')

plt.xticks(rotation=45, ha='right')
plt.savefig('Top 10 Subway Stations by Annual 2022 Ridership Figures', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.pie(borough_total_df.values[:4],autopct='%1.0f%%')
plt.legend(borough_total_df.index[:4],title="Boroughs",loc="center left",bbox_to_anchor=(1, 0, 0.5, 1))
plt.title('Annual Subway Ridership Percentage by Boroughs')
plt.savefig('Annual Subway Ridership Percentage by Boroughs')
plt.show()


In [ ]:
dfmonth=grouped_df.groupby(['station_complex_id','station_complex','borough','latitude','longitude','month'],as_index=False)['ridership'].sum()

In [ ]:
dfmonth.sort_values(by='ridership',ascending=False).head(10)

In [ ]:
top_ten_station_names=dfyear.sort_values(by='ridership',ascending=False).head(10)['station_complex']
top_ten_station_names.values

In [ ]:
dfmonth[dfmonth['station_complex'].isin(top_ten_station_names.values)]
#sort_values(by='ridership',ascending=False).head(10)

In [ ]:
#dfmonth[dfmonth['station_complex'].isin(top_ten_station_names.values)]

In [ ]:
monthtotal=dfmonth.groupby(['month','station_complex_id','station_complex','borough','latitude','longitude'],as_index=False)['ridership'].sum()

In [ ]:
monthtotal[monthtotal['month']=='April']['ridership'].sum()

In [ ]:
monthly_ridership=monthtotal.groupby('month')['ridership'].sum()
month_order=monthnames[1:]
month_order

In [ ]:
monthly_ridership=monthly_ridership.reindex(month_order)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(monthly_ridership.index,monthly_ridership.values,marker='o')
#plt.xlabel('Monthly Ridership Figures')
plt.ylabel('Monthly Ridership Figures (in millions)')
plt.title('NYC Subway Monthly Ridership Trends (2022-2023)')
plt.yticks([6e7, 7e7, 8e7, 9e7, 1e8],['60M', '70M', '80M', '90M', '100M'])
plt.savefig('NYC Subway Monthly Ridership Trends (2022-2023)')
plt.show()

In [ ]:
fig1 = px.density_heatmap(
    x=monthly_ridership.index,   
    y=[""]*len(monthly_ridership),  
    z=monthly_ridership.values,  
    color_continuous_scale="Viridis", 
    labels={'x': 'Month', 'y': '', 'z': 'Ridership'},
)

#fig1.update_layout(
#    title='NYC Subway Monthly Ridership Trends (2022-2023)',
#    xaxis_title='Month',
#    yaxis_title='',
#    yaxis_showticklabels=False,
#    height=500,
#    width=800
#)

fig1.update_layout(
    title={
        'text': 'NYC Subway Monthly Ridership Trends (2022–2023)',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=22, family='Arial', color='darkblue')
    },
    xaxis=dict(
        title='Month',
        titlefont=dict(size=16),
        tickfont=dict(size=14),
        tickangle=45
    ),
    yaxis=dict(
        title='',  # intentionally left blank
        showticklabels=False
    ),
    height=500,
    width=900,
    margin=dict(l=50, r=50, t=80, b=50),
    coloraxis_colorbar=dict(
        title='Ridership',
        ticksuffix='',
        titlefont=dict(size=14),
        tickfont=dict(size=12)
    )
)

fig1.show()

In [ ]:
df2022['time']=df2022['full_datetime'].dt.hour

In [ ]:
hourly_ridership=df2022.groupby('time')['ridership'].sum()
plt.figure(figsize=(14,7))
hourly_ridership.plot(kind='bar',color='#1f77b4',width=0.8,edgecolor='black')
plt.title('Hourly Subway Ridership Distribution (2022)', fontsize=20, fontweight='bold')
plt.xlabel('Hour of Day (24h format)', fontsize=14)
plt.ylabel('Total Entries', fontsize=14)
plt.xticks(rotation=0, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('Hourly Subway Ridership Distribution (2022)',dpi=300, bbox_inches='tight')
plt.tight_layout();

In [ ]:
#changed to mean as i was intented to calculate mean but i originally used sum
hourly_ridership_per_station=df2022.groupby(['station_complex','time','borough','latitude','longitude'])['ridership'].mean().reset_index()

In [ ]:
hourly_ridership_per_station.sort_values(by='ridership',ascending=False)
hourly_ridership_per_station.head(10)

In [ ]:
topten_stations=hourly_ridership_per_station['station_complex'].value_counts().head(10).index
filtered_df = hourly_ridership_per_station[hourly_ridership_per_station['station_complex'].isin(topten_stations)]
filtered_df['time'].value_counts()

In [ ]:
topten_stations=hourly_ridership_per_station['station_complex'].value_counts().head(10).index
filtered_df = hourly_ridership_per_station[hourly_ridership_per_station['station_complex'].isin(topten_stations)]
filtered_df['time'].value_counts()

heatmap_data = filtered_df.pivot_table(
    index='station_complex', 
    columns='time', 
    values='ridership', 
    aggfunc='sum'
)
plt.figure(figsize=(14, 6))
sns.heatmap(heatmap_data, cmap="YlGnBu", linewidths=0.5)
plt.title("Hourly Ridership Heatmap for NYC’s Top 10 Subway Stations")
plt.xlabel('Hour of Day (24h format)', fontsize=14)
plt.ylabel("Top 10 Stations")
plt.savefig("Hourly Ridership Heatmap for NYC’s Top 10 Subway Stations.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
long_heatmap_data = heatmap_data.reset_index().melt(id_vars='station_complex', var_name='time', value_name='ridership')

In [ ]:
fig = px.density_heatmap(
    long_heatmap_data,
    x='time',
    y='station_complex',
    z='ridership',
    nbinsx=24,
    color_continuous_scale='Viridis',
    title='Hourly Ridership Density Heatmap for Top 10 Stations',
    labels={'time': 'Hour', 'station_complex': 'Station', 'ridership': 'Ridership'}
)

fig.update_layout(height=500, width=900)
fig.show()

In [ ]:
import plotly.express as px

fig = px.density_mapbox(
    hourly_ridership_per_station,
    lat='latitude',
    lon='longitude',
    z='ridership',
    radius=15,
    center=dict(lat=40.75, lon=-73.97),
    zoom=10.5,
    mapbox_style="carto-positron",
    animation_frame='time',
    color_continuous_scale="YlOrRd",
    title="Subway Ridership Density Map By Hour"
)

fig.update_layout(height=600, width=900)
fig.show()

In [ ]:
fig = px.bar(
    hourly_ridership_per_station,
    x='time',
    y='ridership',
    color='borough',
    facet_col='borough',
    category_orders={'time': list(range(24))},
    title='Hourly Subway Ridership by Borough',
    labels={'time': 'Hour of Day', 'ridership': 'Total Ridership'}
)

fig.update_layout(height=500, width=1000)
fig.show()

In [ ]:
top3_by_borough = (
    hourly_ridership_per_station
    .groupby(['borough', 'station_complex'])['ridership']
    .sum()
    .reset_index()
    .sort_values(['borough', 'ridership'], ascending=[False, False])
    .groupby('borough')
    .head(3)
)

fig = px.bar(
    top3_by_borough,
    x='ridership',
    y='station_complex',
    orientation='h',
    color='borough',
    title='Top 3 Most Visited Subway Stations in Each NYC Borough (2022)',
    labels={'station_complex': 'Station', 'ridership': 'Total Ridership'}
)

fig.update_layout(xaxis_tickangle=-45, height=600, width=1100)
fig.show()

In [ ]:
fig = px.line(
    dfmonth,  
    x='month',
    y='ridership',
    color='station_complex',
    line_group='station_complex',
    markers=True,
    title='Monthly Ridership Trends for Top NYC Subway Stations (2022)',
    labels={'month': 'Month', 'ridership': 'Monthly Total Ridership','station_complex': 'Subway Station'},
)

fig.update_layout(height=600, width=1100,margin=dict(l=40, r=40, t=60, b=40))
fig.show()

In [ ]:
df2022['day']=df2022['date'].dt.day_name()
df2022['month']=df2022['date'].dt.month_name()

In [ ]:
dfday=df2022.groupby(['day','station_complex','borough','date','month'])['ridership'].sum()
dfday = dfday.reset_index()
dfday.head(5)

In [ ]:
daily_ridership = dfday.groupby('date')['ridership'].mean().reset_index()

fig=px.line(
    daily_ridership,
    x='date',
    y='ridership',
    title='Daily Subway Ridership Over Time',
    labels={'date':'Date','ridership':'Total Ridership'}
)

fig.update_layout(height=500, width=900)
fig.show()


In [ ]:
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_ridership_per_day=dfday.groupby('day')['ridership'].sum().reset_index()
daily_ridership_per_day['day'] = pd.Categorical(daily_ridership_per_day['day'], categories=weekday_order, ordered=True)
daily_ridership_per_day = daily_ridership_per_day.sort_values('day')
fig=px.line(
    daily_ridership_per_day,
    x='day',
    y='ridership',
    title='Total NYC Subway Ridership by Day of the Week (2022)',
    labels={'day':'Day','ridership':'Total Annual Ridership'}
)

fig.update_layout(height=500, width=900)
fig.show()

In [ ]:
daily_ridership_per_day2=dfday.groupby('day')['ridership'].sum().reset_index()
fig=px.box(
    dfday,
    x='day',
    y='ridership',
    title='Daily Subway Ridership BoxPlot',
    labels={'date':'Date','ridership':'Total Ridership'}
)
fig.update_layout(height=500, width=900)
fig.show()

In [ ]:
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_ridership_per_day3=dfday.groupby('day')['ridership'].mean().reset_index()
daily_ridership_per_day3['day'] = pd.Categorical(daily_ridership_per_day3['day'], categories=weekday_order, ordered=True)
daily_ridership_per_day3 = daily_ridership_per_day3.sort_values('day')
fig=px.bar(
    daily_ridership_per_day3,
    x='day',
    y='ridership',
    title='Average Daily Subway Ridership by Day of Week',
    labels={'day':'Day','ridership':'Average Ridership'}
)
fig.update_layout(height=500, width=900)
fig.show()

In [ ]:
dftime=df2022.groupby(['day','station_complex','borough','date','month','time'])['ridership'].sum()
dftime = dftime.reset_index()

In [ ]:
fig = px.line(
    dftime,
    x='time',
    y='ridership',
    color='day',
    animation_frame='date',
    title='Hourly Subway Ridership by Day with Date Slider',
    labels={'time': 'Hour of Day', 'ridership': 'Total Ridership'}
)

fig.update_layout(
    height=500,
    width=900,
    xaxis=dict(tickmode='linear'),
    transition={'duration': 500}
)

fig.show()

In [ ]:
manhattantotalnum.sum()
brooklynTotal=dfyear[dfyear['borough']=='Brooklyn']['ridership']

In [ ]:
hourly_avg

In [ ]:
hourly_avg_day

In [ ]:
queenTotal=dfyear[dfyear['borough']=='Queens']['ridership']
queenTotal.sum()

In [ ]:
manhattanDf=dfyear[dfyear['borough']=='Manhattan']
queensDf=dfyear[dfyear['borough']=='Queens']
brooklynDf=dfyear[dfyear['borough']=='Brooklyn']
bronxDf=dfyear[dfyear['borough']=='Bronx']

In [ ]:
manhattanDf.sort_values(by='ridership',ascending=False)[:5]

In [ ]:
queensDf.sort_values(by='ridership',ascending=False)[:5]

In [ ]:
brooklynDf.sort_values(by='ridership',ascending=False)[:5]

In [ ]:
bronxDf.sort_values(by='ridership',ascending=False)[:5]

In [ ]:
bronxDf['ridership'].sum()

In [ ]:
bronxDf.sort_values?

In [ ]:
manhattanDf
queensDf
brooklynDf
bronxDf

In [ ]:
bronxDf['station_complex'].nunique()

In [ ]:
manhattanDf['station_complex'].nunique()

In [ ]:
queensDf['station_complex'].nunique()

In [ ]:
brooklynDf['station_complex'].nunique()

In [ ]:
manhattanDf['station_complex'].duplicated().sum()

In [ ]:
brooklynDf['station_complex'].duplicated().sum()

In [ ]:
queensDf['station_complex'].duplicated().sum()

In [ ]:
bronxDf['station_complex'].duplicated().sum()

In [ ]:
station_df=df[['station_complex','borough']]
station_df

In [ ]:
station_df.groupby('borough')['station_complex'].sum().reset_index()

In [ ]:
station_df[station_df['borough']=='Manhattan'].nunique()

In [ ]:
station_df[station_df['borough']=='Queens'].nunique()

In [ ]:
dfyear[dfyear['borough']=='Manhattan']['station_complex'].value_counts()

In [ ]:
nyc_subway=pd.read_csv('MTA_Subway_Stations_and_Complexes_20250508.csv')
nyc_subway.info()

In [ ]:
mapping={'Bk':'Brooklyn','M': 'Manhattan','Bx':'Bronx','Q':'Queens'}
nyc_subway['Borough']=nyc_subway['Borough'].replace(mapping)
nyc_subway

In [ ]:
nyc_subway['Borough'].value_counts()

In [ ]:
bronxDf

In [ ]:
nyc_subway[nyc_subway['Borough']=='Bronx']['Station IDs']

In [ ]:
nyc_subway[(nyc_subway['Borough']=='Manhattan')&(nyc_subway['Complex ID']==623)]

In [ ]:
manhattanDf['station_complex_id'].value_counts()

In [ ]:
subset=manhattanDf[manhattanDf['station_complex_id']=='628']
subset.nunique()

In [ ]:
subset_s623=manhattanDf[manhattanDf['station_complex_id']=='623']
subset_s623.nunique()

In [ ]:
nyc_subway[(nyc_subway['Borough']=='Manhattan')&(nyc_subway['Stop Name']=='Canal St')]

In [ ]:
manhattanDf[manhattanDf['station_complex']=='Fulton St (A,C,J,Z,2,3,4,5)']

In [ ]:
manhattanDf[manhattanDf['station_complex']=='Canal St (J,N,Q,R,W,Z,6)']

In [ ]:
manhattanDf['station_complex'].value_counts()
subset['stop_name'].nunique()

In [ ]:
station_df.groupby('borough')['station_complex'].nunique()

In [ ]:
nyc_demographics=pd.read_csv('nycdemog.csv')
nyc_demographics.info()
nyc_demographics.rename(columns={'Gender':'Borough'},inplace=True)
nyc_demographics.info()

In [ ]:
#nyc_demographics=nyc_demographics.set_index('Borough').T
#nyc_demographics=nyc_demographics.iloc[1:]

In [ ]:
nyc_demographics["Total"] = nyc_demographics["Male"] + nyc_demographics["Female"]

In [ ]:
plt.pie(
    nyc_demographics["Total"],
    autopct="%1.1f%%",
    startangle=140
)
plt.legend(nyc_demographics.index, title="Boroughs", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.title('NYC Demographics');

In [ ]:
nyc_demographics[["Male", "Female"]][:4].plot(
    kind="bar", 
    figsize=(10, 6), 
    title="NYC Population by Gender and Borough"
)
plt.xlabel("NYC Boroughs")
plt.ylabel("Population Count (in Millions)")
plt.xticks(rotation=45)
plt.legend(title="Gender")
plt.tight_layout()
plt.savefig("NYC Population by Gender and Borough",dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
plt.pie(
    nyc_demographics[["Male", "Female"]].sum(), 
    labels=["Male", "Female"], 
    autopct="%1.1f%%", 
    startangle=140
)
plt.title("NYC Population Distribution by Gender")
plt.savefig("NYC Population Distribution by Gender",dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
top_ten_station_names = df2022.groupby('station_complex')['ridership'].sum().nlargest(10).index

In [ ]:
top_ten_station_names

In [ ]:
boroughDF=pd.DataFrame(borough_total_df).reset_index()[['borough', 'ridership']].set_index('borough')['ridership']


In [ ]:

boroughDF[:4].reset_index().sort_values(by='ridership',ascending=False)

In [ ]:
daily_ridership_heatmap = dfday.groupby('day')['ridership'].mean().reset_index()

In [ ]:
daily_ridership_heatmap=daily_ridership_heatmap.set_index('day').T

In [ ]:
daily_ridership_heatmap=daily_ridership_heatmap[weekday_order]

In [ ]:
fig = px.imshow(
    daily_ridership_heatmap,
    labels={"x": "Day of Week", "value": "Avg. Ridership"},
    title="Average Daily Subway Ridership by Day of Week"
)

fig.update_layout(height=500, width=900)
fig.show()

In [ ]:
import sys

print("Python version:", sys.version)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

In [ ]:
df.groupby('borough')['station_complex'].count()

In [ ]:
df.info()
station_df.info()

In [ ]:
station_df

plt.pie(
    station_df[['station_complex','borough']].sum(), 
    labels=['borough'], 
    autopct="%1.1f%%", 
    startangle=140
)
plt.title("NYC Population Distribution by Gender")
#plt.savefig("NYC Population Distribution by Gender",dpi=300,bbox_inches='tight')
plt.show()


In [ ]:
plt.pie?

In [ ]:
station_df

In [ ]:
station_complex_out=dfday.groupby(['borough','station_complex'])['ridership'].sum().reset_index()
fig=px.box(
    station_complex_out,
    x='borough',
    y='ridership',
    hover_name='station_complex',
    title='Yearly Subway Ridership BoxPlot',
    labels={'station_complex':'Station Name','ridership':'Total Ridership'}
)
fig.update_layout(height=500, width=900)
st.subheader("Yearly Subway Ridership BoxPlot")
st.plotly_chart(fig_box, use_container_width=True)

In [ ]:
station_complex_out.groupby('borough')

In [ ]:
difference=pd.read_excel('borough_ridership_comparison.xlsx')
difference